In [21]:
import json
import numpy as np
from metric import evaluate, ndcg, average_precision, precision
from collections import defaultdict
from itertools import groupby

In [7]:
def load_judge_file(filepath, scale=int, file_format='ir', reverse=False):
    qd_judge = defaultdict(lambda: defaultdict(lambda: None))
    with open(filepath, 'r') as fp:
        for l in fp:
            if file_format == 'ir':
                q, d, r = l.rstrip().split('\t')
            elif file_format == 'text':
                r, q, d = l.rstrip().split(' ')
            r = scale(r)
            if reverse:
                qd_judge[d][q] = r
            else:
                qd_judge[q][d] = r
    return qd_judge

In [67]:
def rank_judge(json_filename, judgement_filename, file_format='text', reverse=False):
    rel_level = 2
    def relevance_mapper(r):
        if r < 0:
          return 0
        if r >= rel_level:
          return rel_level - 1
        return r
    test_qd_judge = load_judge_file(judgement_filename, file_format=file_format, reverse=reverse)
    for q in test_qd_judge:
        for d in test_qd_judge[q]:
          test_qd_judge[q][d] = relevance_mapper(test_qd_judge[q][d])
    ranks = json.load(open(json_filename, 'r'))
    def dedup(q, docs):
        ndocs = []
        docs_set = set()
        for d in docs:
            if d not in docs_set:
                ndocs.append(d)
                docs_set.add(d)
        return ndocs
    ranks = dict((q, dedup(q, ranks[q])) for q in ranks)
    print('avg #docs per query: {}'.format(np.mean([len(ranks[q]) for q in ranks])))
    for metric, top_k in [('precision', 1), ('precision', 5), ('precision', 10), 
                          ('average_precision', 1), ('average_precision', 5), ('average_precision', 10), ('average_precision', 1000)]:
        scores = evaluate(ranks, test_qd_judge, metric=eval(metric), top_k=top_k)
        avg_score = np.mean(list(scores.values()))
        print('{} {}: {}'.format(metric, top_k, avg_score))

In [79]:
print('matchpyramid')
rank_judge('ranking/matchpyramid/ranking.30.json', 'data/apple_small/test.prep.pointwise', 'text', False)

matchpyramid
avg #docs per query: 366.8448275862069
precision 1: 0.0603448275862069
precision 5: 0.034482758620689655
precision 10: 0.02758620689655173
average_precision 1: 0.0603448275862069
average_precision 5: 0.08994252873563219
average_precision 10: 0.09903530377668308
average_precision 1000: 0.07631416566679959


In [80]:
print('tfidf: use log(tf) || use normalization || use df')
rank_judge('../MatchZoo/experiment/apple_small/ranking/tfidf.json', 'data/apple_small/test.prep.pointwise', 'text', False)

tfidf: use log(tf) || use normalization || use df
avg #docs per query: 366.8448275862069
precision 1: 0.6724137931034483
precision 5: 0.2827586206896553
precision 10: 0.1706896551724138
average_precision 1: 0.6724137931034483
average_precision 5: 0.7228687739463602
average_precision 10: 0.7004436918445539
average_precision 1000: 0.6665067706827957


In [83]:
print('tf: use log(tf) || use normalization')
rank_judge('../MatchZoo/experiment/apple_small/ranking/tf.json', 'data/apple_small/test.prep.pointwise', 'text', False)

tf: use log(tf) || use normalization
avg #docs per query: 366.8448275862069
precision 1: 0.5344827586206896
precision 5: 0.24482758620689654
precision 10: 0.14482758620689656
average_precision 1: 0.5344827586206896
average_precision 5: 0.5933908045977011
average_precision 10: 0.5826855842911878
average_precision 1000: 0.537250900144372


In [84]:
print('tf_nonorm: use log(tf)')
rank_judge('../MatchZoo/experiment/apple_small/ranking/tf_nonorm.json', 'data/apple_small/test.prep.pointwise', 'text', False)

tf_nonorm: use log(tf)
avg #docs per query: 366.8448275862069
precision 1: 0.14655172413793102
precision 5: 0.06724137931034484
precision 10: 0.05689655172413792
average_precision 1: 0.14655172413793102
average_precision 5: 0.18448275862068964
average_precision 10: 0.2046165161466886
average_precision 1000: 0.1755718233759357


In [85]:
print('tf: ')
rank_judge('../MatchZoo/experiment/apple_small/ranking/tf_nonorm_nosublinear.json', 'data/apple_small/test.prep.pointwise', 'text', False)

tf: 
avg #docs per query: 366.8448275862069
precision 1: 0.02586206896551724
precision 5: 0.022413793103448276
precision 10: 0.022413793103448276
average_precision 1: 0.02586206896551724
average_precision 5: 0.044540229885057465
average_precision 10: 0.05430692391899289
average_precision 1000: 0.05942179084335081


In [86]:
print('log tf: ')
rank_judge('ranking/log_tf/ranking.1.json', 'data/apple_small/test.prep.pointwise', 'text', False)

log tf: 
avg #docs per query: 366.8448275862069
precision 1: 0.3103448275862069
precision 5: 0.13103448275862067
precision 10: 0.07672413793103447
average_precision 1: 0.3103448275862069
average_precision 5: 0.3463002873563219
average_precision 10: 0.3433966201423098
average_precision 1000: 0.29068433086339085
